In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, roc_auc_score, mean_absolute_error, accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, LassoCV, LogisticRegression
import category_encoders as ce
from imblearn.over_sampling import SMOTE
from sklearn.utils import class_weight
from xgboost import XGBClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import RidgeClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
import seaborn as sns

%matplotlib inline

C:\Users\yeboa\AppData\Local\Temp\ipykernel_18732\3974177756.py:3: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  import pandas_profiling


ModuleNotFoundError: No module named 'pytorch_tabnet'

In [ ]:
# read data
df = pd.read_csv('train_data.csv')
df2 = pd.read_csv('test_data.csv')
ids = df2['Customer Id']

In [ ]:
df.head()

In [ ]:
def replace_values(val):
    if val == '>=10':
        return 'greater than 10'
    elif val == '   .':
        return 'no data'
    elif int(val) < 5:
        return 'less than 5'
    else:
        return 'greater than 5'

In [ ]:
def preprocess(df, filename):
    # drop ID col 
    df = df.drop('Customer Id', axis=1)

    
    # drop number of windows
    df['NumberOfWindows'] = df['NumberOfWindows'].apply(replace_values)
    
    
    # impute garden based on settlement
    df.loc[df.Settlement == 'U', 'Garden'] = 'V'
    df.loc[df.Settlement != 'U', 'Garden'] = 'O'
    
    
    # impute building dimension
    rural_dim = df[df['Settlement'] == 'R']
    rural_dim_mean = rural_dim["Building Dimension"].mean()
    urban_dim = df[df['Settlement'] == 'U']
    urban_dim_mean = urban_dim["Building Dimension"].mean()
    for i in range(len(df.Settlement)):
        if (df.Settlement[i] == 'U') and pd.isnull(df['Building Dimension'][i]):
            df.at[i, 'Building Dimension'] = urban_dim_mean
        elif (df.Settlement[i] == 'R') and pd.isnull(df['Building Dimension'][i]):
            df.at[i, 'Building Dimension'] = rural_dim_mean
        
        
    #replace occupancy NaN values with mode date
    date_mode = df.Date_of_Occupancy.mode()[0]
    df.Date_of_Occupancy.fillna(date_mode, inplace=True)
    df['Date_of_Occupancy'] = df['Date_of_Occupancy'].astype(int)
    
    
    # Convert the numerical values to string labels for building type
    mapping = {1: 'type1', 2: 'type2', 3: 'type3', 4: 'type4'}
    df['Building_Type'] = df['Building_Type'].map(mapping)
    
    # drop geo code
    df = df.drop('Geo_Code', axis=1)
    
    # save df
    df.to_csv(filename, index=False)

In [ ]:
preprocess(df, 'train_deep_learning.csv')
preprocess(df2, 'test_deep_learning.csv')

In [ ]:
df = pd.read_csv('train_deep_learning.csv')
df2 = pd.read_csv('test_deep_learning.csv')
df.head()